# The More The Merrier (Data Cleaning)

**Description:** This notebook is dedicated to the preprocessing and cleaning of raw data stored in *csv* files using the 
*Pandas* library, specifically focusing on three key datasets for this project.

- **Data:** Datasets to clean:
  - `2017_Entry_Exit.csv`
  - `2017_Average_Housing_Prices_in_London.csv`
  - ` LondonUnderground_Stations_Boroughs.csv `


In [1]:
# importing the necessary libraries
import pandas as pd
import numpy as np

In [2]:
def load_data(filepath):
    '''
    This function loads raw data from a csv file into a pandas dataframe and sets the primary key as the index column
    Args:
        filepath: the raw data's filepath in csv format
    Return:
        The loaded raw data into the pandas dataframe ready to be preprocessed
    '''
    df = pd.read_csv(filepath)
    return df 

### Data Wrangling 
**Description:** After loading the raw data into a dataframe with the function 'load_data', the next step is to wrangle the data. This process involves selecting the relevant columns for analysis, simplifying the column names and adjusting their data types. This process is done using a created class of objects that carries out the aforementioned tasks individually to a dataframe. The resulting dataframes are then ready to be cleaned.

In [3]:
class data_wrangler:
    '''
    This class does the necessary data wrangling such as selecting the relevant columns, setting each column's data type
    and renaming the columns.
    '''
    def __init__(self, df):
        self.df = df
    
    def get_data(self):
        return self.df
    
    def select_cols(self,cols):
        self.df = self.df[cols]
        cleaned_data = data_wrangler(df)
        return self.df

    def adjust_col_dtypes(self,col_dtypes):
        self.df = self.df.astype(col_dtypes)
        cleaned_data = data_wrangler(self.df)
        return self.df
    
    def rename_cols(self,rename):
        self.df.columns = rename
        cleaned_data = data_wrangler(self.df)
        return self.df
    

In [4]:
raw_df = load_data(r'C:\Users\pjxph\Documents\Data Science Projects\The More The Merrier\raw data\2017_Entry_Exit.csv')
# list of all london stations with it's each respective frequency of touch ins and touch outs.

In [5]:
df = data_wrangler(raw_df)
df = df.select_cols(['Station_ID','Station_Name','AnnualEntryExit_Mill'])
df = data_wrangler(df)
df = df.rename_cols(['id','station_name','frequency(mill)'])
df = data_wrangler(df)
df = df.adjust_col_dtypes({
    'id' : int,
    'station_name' : str,
    'frequency(mill)' : np.float64
})
ldn_sta_freq = df

In [6]:
# 2017_Average_hse_price
raw_df = load_data(r'C:\Users\pjxph\Documents/Data Science Projects/The More The Merrier/raw data/2017_Average_Housing_Prices_in_London.csv')

In [7]:
df = data_wrangler(raw_df)
df = df.select_cols(['Area_ID','Area_Name','average_hse_price'])
df = data_wrangler(df)
df = df.rename_cols(['id','name','avg_hse_price'])
df = data_wrangler(df)
df = df.adjust_col_dtypes({
    'id' : str,
    'name' : str,
    'avg_hse_price' : np.float64
})
ldn_hse_price = df 

In [8]:
raw_df = load_data(r'C:\Users\pjxph\Documents/Data Science Projects/The More The Merrier/raw data/LondonUnderground_Stations_Boroughs.csv')

In [9]:
raw_df.head()

,OBJECTID,NAME,NETWORK,Zone,area,Unnamed: 5,Unnamed: 6
0,1,Brent Cross,London Underground,3,Barnet,NaN,London Underground
1,2,Colindale,London Underground,4,Barnet,NaN,London Underground
2,3,Burnt Oak,London Underground,4,Barnet,NaN,London Underground
3,4,Edgware,London Underground,5,Barnet,NaN,London Underground
4,5,Mill Hill East,London Underground,4,Barnet,NaN,London Underground


In [10]:
df = data_wrangler(raw_df)
df = df.select_cols(['OBJECTID','NAME','area'])
df = data_wrangler(df)
df = df.rename_cols(['id','station_name','area_name'])
df = data_wrangler(df)
df = df.adjust_col_dtypes({
    'id' : np.int64,
    'station_name' : str,
    'area_name' : str
})
ldn_bor = df 

In [11]:
# ldn_sta_freq
#ldn_hse_price
# ldn_bor

#### The three **wrangled** dataframes are:
* `ldn_sta_freq`
* `ldn_hse_price`
* `ldn_bor`

Time to clean!

### Data Cleaning
**Description:** After the data wrangling process, the wrangled data needs to be cleaned. The following processes were carried out:
- Ensure the consistency of the station name column in ldn_sta_freq and ldn_bor. 
  - The total number of stations should be the same.
  - Station names should be consistent

- Ensure the consistency of the area_name column in ldn_hse_price and ldn_bor.
  - The total number of london boroughs should be the same.
  - Borough names should be consistent.
- Ensure that each area name is consistent in ldn_bor

In [12]:
# Initial Cleaning Process
class data_cleaner:
    '''
    This class does the necessary data cleaning such as removing duplicates and NaN values for the chosen column.
    '''
    def __init__(self, df):
        self.df = df
        
    def drop_dups(self,columns):
        self.df = ((self.df).drop_duplicates(subset = columns, ignore_index = True))
        return self.df 
    
    def drop_na(self):
        self.df = self.df.dropna()
        return self.df

Firstly, the total number of objects in each dataframe such as london station and boroughs were checked against public records.

The total number of London Underground stations and Boroughs in 2017 were 270 and 32 excluding City Of London respectively.

In [13]:
# raw_data 
print(len(ldn_sta_freq.index))
print(len(ldn_hse_price.index))
print(len(ldn_bor.index))

268
34
270


In [14]:
# To decide which columns to remove duplicates 
ldn_sta_freq.columns

Index(['id', 'station_name', 'frequency(mill)'], dtype='object')

In [15]:
# Partially cleaned ldn_sta_freq
df = data_cleaner(ldn_sta_freq)
df = df.drop_na()
df = data_cleaner(df)
df = df.drop_dups(['station_name'])
df['station_name'] = df['station_name'].str.upper()
ldn_sta_freq = df
print(len(ldn_sta_freq.index))

268


After the initial cleaning process, the total number of underground stations in the ldn_sta_freq dataframe was 268 which is inconsistent with public records. The total number of underground stations in 2017 was 270. The records of the missing stations must be discovered and imported accordingly.ent

In [16]:
# To decide which columns to remove duplicates 
ldn_hse_price.columns

Index(['id', 'name', 'avg_hse_price'], dtype='object')

In [17]:
# Partially cleaned ldn_hse_price
df = data_cleaner(ldn_hse_price)
df = df.drop_na()
df = data_cleaner(df)
df = df.drop_dups(['id','name'])
df['name'] = df['name'].str.upper()
ldn_hse_price = df
print(len(ldn_hse_price.index))

33


After the initial cleaning process, the total number of London boroughs in the ldn_hse_price dataframe was 33 which is consistent with public records. In 2017, the total number of London borough councils was 32 + City Of London.

In [18]:
# To decide which columns to remove duplicates 
ldn_bor.columns

Index(['id', 'station_name', 'area_name'], dtype='object')

In [19]:
### Partially cleaned ldn_bor
df = data_cleaner(ldn_bor)
df = df.drop_dups(['station_name'])
df = data_cleaner(df)
df = df.drop_na()
df['station_name'] = df['station_name'].str.upper()
df['area_name'] = df['area_name'].str.upper()
ldn_bor = df
print(len(ldn_bor.index))
#print(len(ldn_bor['station_name'].unique()))

268


After the initial cleaning process, the total number of London borough in the ldn_bor dataframe was 268 which is consistent with the ldn_hse_price dataframe but inconsistent with public records.

In [20]:
# Test if all station names in ldn_bor dataframe are in ldn_sta_freq dataframe
ldn_bor['station_name'].isin(ldn_sta_freq['station_name']).value_counts()

True    268
Name: station_name, dtype: int64

The test above shows that all stations in **ldn_bor** dataframe are consistent with the dataframe in **ldn_sta_freq**.

Next is to check if all areas in ldn_hse_price dataframe are in ldn_bor. 

In [21]:
a = ldn_hse_price['name'].isin(ldn_bor['area_name'])
print(a.value_counts())

True     27
False     6
Name: name, dtype: int64


In [22]:
b = a == False
ldn_hse_price.loc[b]

,id,name,avg_hse_price
3,E09000004,BEXLEY,330066.0
5,E09000006,BROMLEY,436538.0
7,E09000008,CROYDON,363241.0
20,E09000021,KINGSTON UPON THAMES,487327.0
22,E09000023,LEWISHAM,401025.0
28,E09000029,SUTTON,365567.0


The test showed the above 6 areas in ldn_hse_price_dataframe that are not in ldn_bor. A hypotheses is that these 6 areas do not contain any London Underground Stations. 

After checking against public records, the 6 areas above indeed do not contain any London underground stations. Hence, they can be omitted from the analysis. ldn_hse_price are left with 27 recorded areas.

In [23]:
ldn_hse_price = ldn_hse_price.loc[a]
ldn_hse_price.reset_index(drop = True, inplace = True)

In [24]:
ldn_hse_price['name'].isin(ldn_bor['area_name']).value_counts()

True    27
Name: name, dtype: int64

In [25]:
c = ldn_bor['area_name'].isin(ldn_hse_price['name'])
print(c.value_counts())
#ldn_bor['area_name'].unique()

True     253
False     15
Name: area_name, dtype: int64


In [26]:
d = c == False
ldn_borx = ldn_bor.loc[d]

The test above showed that these 15 stations and respective area in the ldn_bor dataframe that are not contained inside the ldn_bor dataframe. These areas correspond to district councils. They can be added to the ldn_hse_price dataframe with gov.uk house price index data. The dataframe name is changed from ldn_bor to ldn_council

In [27]:
ldn_council = ldn_bor

In [28]:
raw_df = load_data(r'C:\Users\pjxph\Documents\Data Science Projects\The More The Merrier\raw data\England-annual-price-change-by-local-authority-2017-12 (1).csv')

In [29]:
raw_df['December 2017'] = raw_df['December 2017'].str.replace('�', '')
df = data_wrangler(raw_df)
df = df.select_cols(['Local authorities','December 2017'])
df = data_wrangler(df)
df = df.rename_cols(['name','avg_hse_price'])
df = data_wrangler(df)
df = df.adjust_col_dtypes({
    'name' : str,
    'avg_hse_price' : np.float64
})
df['name'] = df['name'].str.upper()
ldn_hse_price_x = df
ldn_hse_price_x

,name,avg_hse_price
0,ADUR,306921.0
1,ALLERDALE,149657.0
2,AMBER VALLEY,170198.0
3,ARUN,288820.0
4,ASHFIELD,135115.0
...,...,...
348,WYCOMBE,405071.0
349,WYRE,150409.0
350,WYRE FOREST,184840.0
351,YORK,242125.0


In [30]:
xbor = ldn_bor.loc[d]['area_name'].unique()

In [31]:
ldn_hse_price_x = ldn_hse_price_x.loc[ldn_hse_price_x['name'].isin(xbor)]
ldn_hse_price_x = ldn_hse_price_x.assign(id = ['E09000034','E09000035','E09000036','E09000037'])

In [32]:
ldn_hse_price = pd.concat([ldn_hse_price, ldn_hse_price_x])
ldn_hse_price.reset_index(drop = True, inplace = True)

In [33]:
ldn_hse_price

,id,name,avg_hse_price
0,E09000001,CITY OF LONDON,820305.0
1,E09000002,BARKING AND DAGENHAM,282441.0
2,E09000003,BARNET,532924.0
3,E09000005,BRENT,472373.0
4,E09000007,CAMDEN,856070.0
5,E09000009,EALING,489364.0
6,E09000010,ENFIELD,391874.0
7,E09000011,GREENWICH,376668.0
8,E09000012,HACKNEY,561445.0
9,E09000013,HAMMERSMITH AND FULHAM,755421.0


In [34]:
ldn_bor = ldn_bor.loc[c]
#ldn_bor.reset_index(inplace = True)

In [35]:
ldn_bor['area_name'].isin(ldn_hse_price['name']).value_counts()

True    253
Name: area_name, dtype: int64

In [36]:
# Station Name Finder 
ldn_bor.loc[ldn_bor['station_name'] == 'Brent Cross']

,id,station_name,area_name
